In [1]:
#modules required
import numpy as np
import time
import scipy.signal as s #for our 2d convolution explained below
import panel as pn
import pandas as pd
import random
pn.extension('deckgl', design='material', sizing_mode="stretch_width")

In [2]:
points = {
    '@@type': 'PointCloudLayer',
    'data': [],
    #'coordinateOrigin': [0, 0],
    'getColor': '@@=color',
    'getPosition': '@@=position',
    'pointSize': 6.8,
    'id': 'pointcloudlayer'
}

board_json = {
     "initialViewState": {
        "bearing": 0,
        "latitude": .1,
        "longitude": 0,
        "pitch": 0,
        "zoom": 6.92
    },
    "layers": [points],
    "mapStyle": "",
    "views": [
        {
            "@@type": "MapView",
            "controller": True
        }
    ]
}

In [3]:

#lookup for speed (alive_status,alive_neighbor_sum) : result
rules_look_up = {(1,0): 0, (1,1):0, (1,2):1, (1,3):1, (1,4):0, (1,5):0, (1,6):0, (1,7):0, (1,8):0, 
              (0,0): 0, (0,1):0, (0,2):0, (0,3):1, (0,4):0, (0,5):0, (0,6):0, (0,7):0, (0,8):0}


#we'll use this kernel to get our neighbor values using a convolution  
kernel =[[1,1,1],
         [1,0,1],
         [1,1,1]]

#let's set up some defaults for our game, we'll be able to change most of these with panel widgets we'll see at the end
tick_delay = 200 # our tick delay in miliseconds
world_dim = 10 # our world is square world_dimxworld_dim
outside_boundary = 1 # cells outside boundary considered what, I kept it at 1 to allow the bounadires to be life giving :) 


In [4]:
#lookup our rules with our dictionary by using our world matrix and the convoluted matrix to get the new alive status of each 
def lookup(world_status, alive_neighbors):
    global rules_look_up
    return rules_look_up[(world_status, alive_neighbors)]
vlookup = np.vectorize(lookup) # this is required so that it will run on all cells (basically act like a for loop) of what is passed in

In [5]:
def game_tick(world_view):
    time_tick = time.perf_counter() #perf_counter is more accurate than time.time()
    global outside_boundary
    stable=True # keep track of if the population (# of 1's) changes or stays the same
    world_buffer = s.convolve2d(world_view, kernel,mode='same',boundary='fill',fillvalue = outside_boundary) #get the number of neighbors
    world_buffer = vlookup(world_view, world_buffer) # apply all the rules and store teh new state in teh world_buffer
    stable = (np.count_nonzero(world_view)==np.count_nonzero(world_buffer))#stable if the nonzero before and after ==  same population   
    time_tick = np.round(1 / (time.perf_counter()-time_tick)) # get the number this can be run per second
    return (world_buffer,f"** population {np.count_nonzero(world_view)} ** stable {stable} ** game_tick Speed {time_tick} calls per sec ") #return the modified world_buffer with stats

In [6]:
SCALING_FACTOR = 10.0

alive_rgba = (209, 159, 159, 255) 
dead_rgba = (179,255,179,255)


def init_world(dim, random=True,points=None): # numpy init 
    if random:
        create_world = np.random.randint(0,2,size=(dim,dim))
    else:
        create_world = np.zeros((dim,dim))
        if points is not None:
            for p in points: #if we pass intial points to try some inital conditions ourselves 
                if 0<=p[0]<create_world.shape[0] and 0<=p[1]<create_world.shape[1]:# make sure we're inside the world before we try to make alive  
                    create_world[p[0],p[1]]=1 
    return create_world #return our new world
    
def numpy_to_deckgl_data_fast(world): # make it able to go into dictionary/json
    # Get the x and y positions from the shape of our world numpy array
    world = np.flip(world, axis=0) # orient it with the latitude so latitutde starts at 0 at the top and increases as you go down 
    x,y= np.meshgrid(np.arange(world.shape[0]), np.arange(world.shape[1]), indexing='xy') # x is columns i is rows
    position = np.dstack([x, y]).reshape(-1, 2)/SCALING_FACTOR
    # Create the color array
    color = np.where(world.flatten()[:,None] == 0, list(dead_rgba), list(alive_rgba))
    # return a dataframe yeah it needs it 
    return pd.DataFrame({'position': position.tolist(), 'color': color.tolist()})
    

In [7]:
mainview = pn.pane.DeckGL(board_json, height=200,width=800)
def update_point_size(event):
    points['pointSize']=event.new
    mainview.param.trigger('object')
np_world = init_world(2)
print(np_world)
dta=numpy_to_deckgl_data_fast(np_world)
print('')
print(dta)
points['data'] = dta
point_size_slider = pn.widgets.FloatSlider(name='point_size_slider',start=0.01, end=50.0,step=.0005,value=7.8)
point_size_slider.param.watch(update_point_size, 'value')
pn.Column(point_size_slider,mainview).servable()

[[1 1]
 [0 0]]

     position                 color
0  [0.0, 0.0]  [179, 255, 179, 255]
1  [0.1, 0.0]  [179, 255, 179, 255]
2  [0.0, 0.1]  [209, 159, 159, 255]
3  [0.1, 0.1]  [209, 159, 159, 255]


Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [0] FloatSlider(design=<class 'panel.theme.materi..., end=50.0, name='point_size_slider', sizing_mode='stretch_width', start=0.01, step=0.0005, value=7.8)
    [1] DeckGL(dict, design=<class 'panel.theme.materi..., height=200, sizing_mode='fixed', width=800)

In [8]:
elapsed = 0.0
iterations = 150
#for n in range(iterations):
world = init_world(3,random=True,points=None)
print(world)
tick = time.perf_counter()
data = numpy_to_deckgl_data_fast(world)
timing = time.perf_counter() - tick
points['data']=data
#mainview.param.trigger('object')
elapsed += timing

[[1 0 1]
 [1 0 0]
 [0 0 1]]


In [9]:
# all the button callbacks 
def update_zoom(event):
    board_json['initialViewState']['zoom']=event.new
    mainview.param.trigger('object')
    #mainview.param.trigger('object') # trigger an update if we change the data


def update_point_size(event):
    points['pointSize']=event.new
    mainview.param.trigger('object')

def toggle_periodic_callback(event): # turn on and off the game loop
    if event.new:
        periodic_toggle.name = 'Stop'
        periodic_toggle.button_type = 'warning'
        periodic_cb.start()
    else:
        periodic_toggle.name = 'Run'
        periodic_toggle.button_type = 'primary'
        periodic_cb.stop()

def update_boundary(event):
    global outside_boundary
    outside_boundary=event.new

def game_loop(event=None):
    global world
    (world,status) = game_tick(world)     
    points['data']=numpy_to_deckgl_data_fast(world.copy())
    mainview.param.trigger('object') # trigger an update if we change the data
    static_text.value=f"{status} "

def reset_button(event=None):
    global world
    global outside_boundary
    outside_boundary=boundary_switch.value
    world = init_world(world_dim_slider.value,random=random_switch.value,points=None)
    points['data']=numpy_to_deckgl_data_fast(world.copy())
    mainview.param.trigger('object') # trigger an update if we change the data

def convert_color_hex(color):
    return f"#{color[0]:02X}{color[1]:02X}{color[2]:02X}"

def update_period(event):
    periodic_cb.period = event.new

def update_alive_color(event):
    # Convert hex to RGBA tuple
    hex_color = event.new
    r = int(hex_color[1:3], 16)
    g = int(hex_color[3:5], 16)
    b = int(hex_color[5:7], 16)
    a = 255 
    global alive_rgba
    alive_rgba = (r,g,b,255)

def update_dead_color(event):
    # Convert hex to RGBA tuple
    hex_color = event.new
    r = int(hex_color[1:3], 16)
    g = int(hex_color[3:5], 16)
    b = int(hex_color[5:7], 16)
    a = 255 
    global dead_rgba
    dead_rgba = (r,g,b,255)

In [10]:
zoom_slider = pn.widgets.FloatSlider(name='zoom_slider',start=0.0, end=15.0,step=.00001,value=6.07)
zoom_slider.param.watch(update_zoom, 'value')

point_size_slider = pn.widgets.FloatSlider(name='point_size_slider',start=0.01, end=50.0,step=.0005,value=6.8)
point_size_slider.param.watch(update_point_size, 'value')


tick_delay_slider = pn.widgets.FloatSlider(name='tick_delay',start=1, end=2000,step=1,value=tick_delay)
tick_delay_slider.param.watch(update_period, 'value')

world_dim_slider = pn.widgets.IntSlider(name='world_size',start=4,end=500,step=1,value=world_dim)
world_dim_slider.param.watch(reset_button, 'value')

static_text = pn.widgets.StaticText(name='World Status', value='', width=500)

str_pane1 = pn.pane.Str('random world?', width=110)
random_switch = pn.widgets.Switch(name='Switch1', value=True, width=50)

str_pane2 = pn.pane.Str('boundaries alive?', width=110)
boundary_switch = pn.widgets.Switch(name='Switch2', value=True, width=50)
boundary_switch.param.watch(update_boundary, 'value')


button = pn.widgets.Button(name='Step', width = 50)
button.on_click(game_loop) # link up our button to trigger the game_loop function we made
                        
periodic_toggle = pn.widgets.Toggle(
    name='Run', value=False, button_type='primary', align='end', width=50
)

reset = pn.widgets.Button(name='Reset', button_type='warning', width=60, align='end')
reset.on_click(reset_button) # link up our button to trigger the game_loop function we made

mainview = pn.pane.DeckGL(board_json, height=400,width=600)
mainview.param.trigger('object')


alive_color_picker = pn.widgets.ColorPicker(name='Alive', value=convert_color_hex(alive_rgba), width=50)
alive_color_picker.param.watch(update_alive_color, "value")

dead_color_picker = pn.widgets.ColorPicker(name='Dead', value=convert_color_hex(dead_rgba), width=50)
dead_color_picker.param.watch(update_dead_color, "value")

periodic_toggle.param.watch(toggle_periodic_callback, 'value') # fire toggle_periodic_callback if the param changes
periodic_cb = pn.state.add_periodic_callback(game_loop, start=False, period=int(tick_delay_slider.value))
reset_button()
pn.Column(
    pn.Column(pn.Row(tick_delay_slider,world_dim_slider,point_size_slider),
    pn.Row(alive_color_picker,dead_color_picker,str_pane1,random_switch,str_pane2,boundary_switch)),

    pn.Column(
    pn.Row(static_text,button,periodic_toggle,reset)),
    
    pn.Row(pn.Column(mainview),pn.Column(mainview))
).servable()


Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [0] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [0] FloatSlider(design=<class 'panel.theme.materi..., end=2000, name='tick_delay', sizing_mode='stretch_width', start=1, step=1, value=200)
            [1] IntSlider(design=<class 'panel.theme.materi..., end=500, name='world_size', sizing_mode='stretch_width', start=4, value=10)
            [2] FloatSlider(design=<class 'panel.theme.materi..., end=50.0, name='point_size_slider', sizing_mode='stretch_width', start=0.01, step=0.0005, value=6.8)
        [1] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [0] ColorPicker(design=<class 'panel.theme.materi..., name='Alive', value='#D19F9F', width=50)
            [1] ColorPicker(design=<class 'panel.theme.materi..., name='Dead', value='#B3FFB3', width=50)
            [2] Str(str, design=<class 'panel.theme.materi..., width=110)
            [3] Switch(design=<class 'panel.theme.materi..., value=True, width=50)
            [4] Str(str, design=<class 'panel.theme.materi..., width=110)
            [5] Switch(design=<class 'panel.theme.materi..., value=True, width=50)
    [1] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [0] StaticText(design=<class 'panel.theme.materi..., name='World Status', width=500)
            [1] Button(design=<class 'panel.theme.materi..., name='Step', width=50)
            [2] Toggle(align='end', button_type='primary', design=<class 'panel.theme.materi..., name='Run', width=50)
            [3] Button(align='end', button_type='warning', design=<class 'panel.theme.materi..., name='Reset', width=60)
    [2] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [0] DeckGL(dict, design=<class 'panel.theme.materi..., height=400, sizing_mode='fixed', width=600)
        [1] Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
            [0] DeckGL(dict, design=<class 'panel.theme.materi..., height=400, sizing_mode='fixed', width=600)